In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip install catboost

     |████████████████████████████████| 65.9MB 53kB/s 


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score


from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, cross_validate
from lightgbm import LGBMClassifier
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor, RandomForestClassifier, RandomForestRegressor, AdaBoostClassifier, AdaBoostRegressor, ExtraTreesClassifier, ExtraTreesRegressor, BaggingRegressor, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
import numpy as np
from sklearn.svm import SVC, SVR
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
class Tuner:
  def __init__(self, X_train, y_train, algo_type='binary', search_opt='randomized', scoring_fit='accuracy', n_iteration=100, cv=5):
    self.X_train = X_train
    self.y_train = y_train
    self.algo_type = algo_type
    self.search_opt = search_opt
    self.scoring_fit = scoring_fit
    self.n_iteration = n_iteration
    self.cv = cv

  def randomized_opt(self, model, parameter_grid):
    randomized_search = RandomizedSearchCV(
                        estimator=model,
                        param_distributions=parameter_grid, 
                        cv=self.cv,
                        n_iter=self.n_iteration,
                        n_jobs=-1, 
                        scoring=self.scoring_fit,
                        verbose=2)
    fitted_model = randomized_search.fit(self.X_train, self.y_train)
    return fitted_model

  def grid_opt(self, model, parameter_grid):
    grid_search = GridSearchCV(
                        estimator=model,
                        param_grid=parameter_grid, 
                        cv=self.cv,
                        n_jobs=-1, 
                        scoring=self.scoring_fit,
                        verbose=2)
    fitted_model = grid_search.fit(self.X_train, self.y_train)
    return fitted_model

  def bayesian_opt(self, model, parameter_grid):
    randomized_search = RandomizedSearchCV(
                        estimator=model,
                        param_distributions=parameter_grid, 
                        cv=self.cv,
                        n_iter=self.n_iteration,
                        n_jobs=-1, 
                        scoring=self.scoring_fit,
                        verbose=2)
    fitted_model = randomized_search.fit(self.X_train, self.y_train)
    return fitted_model

  def print_info(self, model):
    print(model.best_score_)
    print(model.best_params_)

  def fit_LGBM(self):
      
    if self.algo_type == 'binary':
      objective = LGBMClassifier(objective='binary')
    elif self.algo_type == 'multiclass':
      objective = LGBMClassifier(objective='multiclass')
    elif self.algo_type =='regression':
      objective = LGBMClassifier(objective='regression')
      
    print('Optimizing LGBM algorithm...')
    param_grid = {
        'num_leaves': [5, 10, 20, 30, 50, 100, 200],
        'min_data_in_leaf': [10, 20, 30, 50, 100, 200, 500],
        "learning_rate": [0.01, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30],
        'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'min_child_weight': [0.001, 0.01, 0.1, 0.5, 1.0, 3.0, 5.0, 7.0, 10.0],
        'reg_lambda': [0, 0.1, 1.0, 1.1, 1.5],
        'reg_alpha': [0, 0.1, 1.0, 1.1, 1.5],
        'n_estimators': [30, 70, 100, 150, 400, 700]
    }

    if self.search_opt == 'grid':
      model = self.grid_opt(objective, param_grid)
      self.print_info(model)
      return model
    elif self.search_opt == 'bayesian':
      model = self.bayesian_opt(objective, param_grid)
      self.print_info(model)
      return model
    elif self.search_opt == 'randomized':
      model = self.randomized_opt(objective, param_grid)
      self.print_info(model)
      return model
    elif self.search_opt == 'none':
      scores = cross_validate(objective, self.X_train, self.y_train, cv=self.cv, return_estimator=True)
      print(scores['test_score'].max())
      return scores['estimator'][scores['test_score'].argmax()]
    else:
      raise ValueError('please provide a correct search optimization technique') 

  
  def fit_XGB(self):
      
    if self.algo_type == 'binary':
      objective = xgb.XGBClassifier(objective='binary:logistic')
    elif self.algo_type == 'multiclass':
      objective = xgb.XGBClassifier(objective='multiclass:softmax')
    elif self.algo_type =='regression':
      objective = xgb.XGBClassifier(objective='regression:squarederror')

    print('Optimizing XGB algorithm...')
    param_grid = {
        'max_depth': [None, 1, 3, 6, 10, 15, 20],
        "learning_rate": [0.01, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ],
        'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bylevel': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'min_child_weight': [0.5, 1.0, 3.0, 5.0, 7.0, 10.0],
        'gamma': [0, 0.25, 0.5, 1.0],
        'reg_lambda': [0, 0.1, 1.0, 5.0, 10.0, 50.0, 100.0],
        'n_estimators': [30, 70, 100, 150, 250, 400, 700]
    }

    if self.search_opt == 'grid':
      model = self.grid_opt(objective, param_grid)
      self.print_info(model)
      return model
    elif self.search_opt == 'bayesian':
      model = self.bayesian_opt(objective, param_grid)
      self.print_info(model)
      return model
    elif self.search_opt == 'randomized':
      model = self.randomized_opt(objective, param_grid)
      self.print_info(model)
      return model
    elif self.search_opt == 'none':
      scores = cross_validate(objective, self.X_train, self.y_train, cv=self.cv, return_estimator=True)
      print(scores['test_score'].max())
      return scores['estimator'][scores['test_score'].argmax()]
    else:
      raise ValueError('please provide a correct search optimization technique') 

  def fit_CatBoost(self):
      
    if self.algo_type == 'binary' or self.algo_type == 'multiclass':
      objective = CatBoostClassifier()
    elif self.algo_type =='regression':
      objective = CatBoostRegressor()

    print('Optimizing CatBoost algorithm...')

    param_grid = {'depth':[3,1,2,6,4,5,7,8,9,10],
              'iterations':[250,100,500,1000],
              "learning_rate": [0.01, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ],
              'l2_leaf_reg':[3,1,5,10,100],
              'border_count':[32,5,10,20,50,100,200],
              'ctr_border_count':[50,5,10,20,100,200],
              }

    if self.search_opt == 'grid':
      model = self.grid_opt(objective, param_grid)
      self.print_info(model)
      return model
    elif self.search_opt == 'bayesian':
      model = self.bayesian_opt(objective, param_grid)
      self.print_info(model)
      return model
    elif self.search_opt == 'randomized':
      model = self.randomized_opt(objective, param_grid)
      self.print_info(model)
      return model
    elif self.search_opt == 'none':
      scores = cross_validate(objective, self.X_train, self.y_train, cv=self.cv, return_estimator=True)
      print(scores['test_score'].max())
      return scores['estimator'][scores['test_score'].argmax()]
    else:
      raise ValueError('please provide a correct search optimization technique') 

  def fit_GBM(self):
      
    if self.algo_type == 'binary' or self.algo_type == 'multiclass':
      objective = GradientBoostingClassifier()
    elif self.algo_type =='regression':
      objective = GradientBoostingRegressior()

    print('Optimizing Gradient Boosting algorithm...')
    param_grid = {
        'max_depth': [None, 1, 3, 6, 10, 15, 20],
        "learning_rate": [0.01, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30],
        "min_samples_split": [0.1, 0.5, 1, 2, 3, 5, 10],
        "min_samples_leaf": [0.1, 0.5, 1, 2, 3, 5, 10],
        'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'n_estimators': [30, 70, 100, 150, 250, 400, 700],
        "max_features":["auto","log2","sqrt"],
        "criterion": ["friedman_mse",  "mae", "mse"]
    }

    if self.search_opt == 'grid':
      model = self.grid_opt(objective, param_grid)
      self.print_info(model)
      return model
    elif self.search_opt == 'bayesian':
      model = self.bayesian_opt(objective, param_grid)
      self.print_info(model)
      return model
    elif self.search_opt == 'randomized':
      model = self.randomized_opt(objective, param_grid)
      self.print_info(model)
      return model
    elif self.search_opt == 'none':
      scores = cross_validate(objective, self.X_train, self.y_train, cv=self.cv, return_estimator=True)
      print(scores['test_score'].max())
      return scores['estimator'][scores['test_score'].argmax()]
    else:
      raise ValueError('please provide a correct search optimization technique') 

  def fit_RF(self):
      
    if self.algo_type == 'binary' or self.algo_type == 'multiclass':
      objective = RandomForestClassifier()
      param_grid = {
          'max_depth': [None, 1, 3, 6, 10, 15, 20],
          'bootstrap': [True, False],
          "min_samples_split": [0.1, 0.5, 1, 2, 3, 5, 10],
          "min_samples_leaf": [0.1, 0.5, 1, 2, 3, 5, 10],
          'n_estimators': [30, 70, 100, 150, 250, 400, 700],
          "max_features":["auto","log2","sqrt"],
          "criterion": ["gini", "entropy"]
      }
    elif self.algo_type =='regression':
      objective = RandomForestRegressor()
      param_grid = {
          'max_depth': [None, 1, 3, 6, 10, 15, 20],
          'bootstrap': [True, False],
          "min_samples_split": [0.1, 0.5, 1, 2, 3, 5, 10],
          "min_samples_leaf": [0.1, 0.5, 1, 2, 3, 5, 10],
          'n_estimators': [30, 70, 100, 150, 250, 400, 700],
          "max_features":["auto","log2","sqrt"],
          "criterion": ["mse", "mae"]
      }
    print('Optimizing Random Forest algorithm...')


    if self.search_opt == 'grid':
      model = self.grid_opt(objective, param_grid)
      self.print_info(model)
      return model
    elif self.search_opt == 'bayesian':
      model = self.bayesian_opt(objective, param_grid)
      self.print_info(model)
      return model
    elif self.search_opt == 'randomized':
      model = self.randomized_opt(objective, param_grid)
      self.print_info(model)
      return model
    elif self.search_opt == 'none':
      scores = cross_validate(objective, self.X_train, self.y_train, cv=self.cv, return_estimator=True)
      print(scores['test_score'].max())
      return scores['estimator'][scores['test_score'].argmax()]
    else:
      raise ValueError('please provide a correct search optimization technique') 

  def fit_ExtraTrees(self):
      
    if self.algo_type == 'binary' or self.algo_type == 'multiclass':
      objective = ExtraTreesClassifier()
      param_grid = {
          'max_depth': [None, 1, 3, 6, 10, 15, 20],
          'bootstrap': [True, False],
          "min_samples_split": [0.1, 0.5, 1, 2, 3, 5, 10],
          "min_samples_leaf": [0.1, 0.5, 1, 2, 3, 5, 10],
          'n_estimators': [30, 70, 100, 150, 250, 400, 700],
          "max_features":["auto","log2","sqrt"],
          "criterion": ["gini", "entropy"]
      }
    elif self.algo_type =='regression':
      objective = ExtraTreesRegressor()
      param_grid = {
          'max_depth': [None, 1, 3, 6, 10, 15, 20],
          'bootstrap': [True, False],
          "min_samples_split": [0.1, 0.5, 1, 2, 3, 5, 10],
          "min_samples_leaf": [0.1, 0.5, 1, 2, 3, 5, 10],
          'n_estimators': [30, 70, 100, 150, 250, 400, 700],
          "max_features":["auto","log2","sqrt"],
          "criterion": ["mae", "mse"]
      }

    print('Optimizing ExtraTrees algorithm...')


    if self.search_opt == 'grid':
      model = self.grid_opt(objective, param_grid)
      self.print_info(model)
      return model
    elif self.search_opt == 'bayesian':
      model = self.bayesian_opt(objective, param_grid)
      self.print_info(model)
      return model
    elif self.search_opt == 'randomized':
      model = self.randomized_opt(objective, param_grid)
      self.print_info(model)
      return model
    elif self.search_opt == 'none':
      scores = cross_validate(objective, self.X_train, self.y_train, cv=self.cv, return_estimator=True)
      print(scores['test_score'].max())
      return scores['estimator'][scores['test_score'].argmax()]
    else:
      raise ValueError('please provide a correct search optimization technique') 

  def fit_DT(self):
      
    if self.algo_type == 'binary' or self.algo_type == 'multiclass':
      objective = DecisionTreeClassifier()
      param_grid = {
          'max_depth': [None, 1, 3, 6, 10, 15, 20],
          "min_samples_split": [0.1, 0.5, 1, 2, 3, 5, 10],
          "min_samples_leaf": [0.1, 0.5, 1, 2, 3, 5, 10],
          "max_features":["auto","log2","sqrt"],
          "criterion": ["gini", "entropy"]
      }
    elif self.algo_type =='regression':
      objective = DecisionTreeRegressor()
      param_grid = {
          'max_depth': [None, 1, 3, 6, 10, 15, 20],
          "min_samples_split": [0.1, 0.5, 1, 2, 3, 5, 10],
          "min_samples_leaf": [0.1, 0.5, 1, 2, 3, 5, 10],
          "max_features":["auto","log2","sqrt"],
          "criterion": ["friedman_mse",  "mae", "mse"]
      }

    print('Optimizing Decision Tree algorithm...')

    if self.search_opt == 'grid':
      model = self.grid_opt(objective, param_grid)
      self.print_info(model)
      return model
    elif self.search_opt == 'bayesian':
      model = self.bayesian_opt(objective, param_grid)
      self.print_info(model)
      return model
    elif self.search_opt == 'randomized':
      model = self.randomized_opt(objective, param_grid)
      self.print_info(model)
      return model
    elif self.search_opt == 'none':
      scores = cross_validate(objective, self.X_train, self.y_train, cv=self.cv, return_estimator=True)
      print(scores['test_score'].max())
      return scores['estimator'][scores['test_score'].argmax()]
    else:
      raise ValueError('please provide a correct search optimization technique') 

  def fit_AdaBoost(self):
      
    if self.algo_type == 'binary' or self.algo_type == 'multiclass':
      objective = AdaBoostClassifier()
      param_grid = {
                  'n_estimators': [30, 70, 100, 150, 250, 400, 700],
                  "learning_rate": [0.01, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30]
                  }
    elif self.algo_type =='regression':
      objective = AdaBoostRegressor()
      param_grid = {
                  'n_estimators': [30, 70, 100, 150, 250, 400, 700],
                  "learning_rate": [0.01, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30],
                  'loss' : ['linear', 'square', 'exponential']
                  }
    print('Optimizing AdaBoost algorithm...')

    if self.search_opt == 'grid':
      model = self.grid_opt(objective, param_grid)
      self.print_info(model)
      return model
    elif self.search_opt == 'bayesian':
      model = self.bayesian_opt(objective, param_grid)
      self.print_info(model)
      return model
    elif self.search_opt == 'randomized':
      model = self.randomized_opt(objective, param_grid)
      self.print_info(model)
      return model
    elif self.search_opt == 'none':
      scores = cross_validate(objective, self.X_train, self.y_train, cv=self.cv, return_estimator=True)
      print(scores['test_score'].max())
      return scores['estimator'][scores['test_score'].argmax()]
    else:
      raise ValueError('please provide a correct search optimization technique') 

  def fit_SVM(self):
      
    param_grid = {'C': [0.1, 1, 10, 30, 40, 100, 1000],
                  'gamma': [1, 0.1, 0.01, 0.001],
                  'kernel': ['rbf', 'poly', 'sigmoid']
                  }
    if self.algo_type == 'binary' or self.algo_type == 'multiclass':
      objective = SVC()

    elif self.algo_type =='regression':
      objective = SVR()

    print('Optimizing SVM algorithm...')


    if self.search_opt == 'grid':
      model = self.grid_opt(objective, param_grid)
      self.print_info(model)
      return model
    elif self.search_opt == 'bayesian':
      model = self.bayesian_opt(objective, param_grid)
      self.print_info(model)
      return model
    elif self.search_opt == 'randomized':
      model = self.randomized_opt(objective, param_grid)
      self.print_info(model)
      return model
    elif self.search_opt == 'none':
      scores = cross_validate(objective, self.X_train, self.y_train, cv=self.cv, return_estimator=True)
      print(scores['test_score'].max())
      return scores['estimator'][scores['test_score'].argmax()]
    else:
      raise ValueError('please provide a correct search optimization technique') 

  def fit_LogR(self):

    print('Optimizing Logistic Regression algorithm...')
    param_grid = {
        'penalty' : ['l1', 'l2'],
        'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]
    }
    objective = LogisticRegression()

    if self.search_opt == 'grid':
      model = self.grid_opt(objective, param_grid)
      self.print_info(model)
      return model
    elif self.search_opt == 'bayesian':
      model = self.bayesian_opt(objective, param_grid)
      self.print_info(model)
      return model
    elif self.search_opt == 'randomized':
      model = self.randomized_opt(objective, param_grid)
      self.print_info(model)
      return model
    elif self.search_opt == 'none':
      scores = cross_validate(objective, self.X_train, self.y_train, cv=self.cv, return_estimator=True)
      print(scores['test_score'].max())
      return scores['estimator'][scores['test_score'].argmax()]
    else:
      raise ValueError('please provide a correct search optimization technique') 

  def fit_Bagging(self):

    print('Optimizing Bagging algorithm...')
    param_grid = {
        'n_estimators': [30, 70, 100, 150, 250, 400, 700],
        'max_samples': [0.05, 0.1, 0.2, 0.5, 1]
    }
    if self.algo_type == 'binary' or self.algo_type == 'multiclass':
      objective = BaggingClassifier()
    elif self.algo_type =='regression':
      objective = BaggingRegressor()


    if self.search_opt == 'grid':
      model = self.grid_opt(objective, param_grid)
      self.print_info(model)
      return model
    elif self.search_opt == 'bayesian':
      model = self.bayesian_opt(objective, param_grid)
      self.print_info(model)
      return model
    elif self.search_opt == 'randomized':
      model = self.randomized_opt(objective, param_grid)
      self.print_info(model)
      return model
    elif self.search_opt == 'none':
      scores = cross_validate(objective, self.X_train, self.y_train, cv=self.cv, return_estimator=True)
      print(scores['test_score'].max())
      return scores['estimator'][scores['test_score'].argmax()]
    else:
      raise ValueError('please provide a correct search optimization technique') 

  def fit_ANN(self):

    param_grid = {
                    'hidden_layer_sizes': [(3,), (6,), (10,), (15,), (20,), (30,), (40,), (50,), (100,), (150,), (200,),
                                           (3,3,), (6,6,), (10,10,), (15,15,), (20,20,), (30,30,), (40,40,), (50,50,), (100,100,), (150,150,), (200,200),],
                    'learning_rate_init': [0.01, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30],
                    'learning_rate': ['constant', 'invscaling', 'adaptive'],
                    'batch_size': [25, 50, 100, 200, 300],
                    'solver': ['lbfgs', 'sgd', 'adam'],
                    'activation': ['identity', 'logistic', 'tanh', 'relu'],
                    'alpha': [0, 0.001, 0.01, 0.1, 0.5, 1.0, 1.1, 1.5],
                  }      

    if self.algo_type == 'binary' or self.algo_type == 'multiclass':
      objective = MLPClassifier(early_stopping=True)

    elif self.algo_type =='regression':
      objective = MLPRegressor(early_stopping=True)

    print('Optimizing Artificial Neural Network...')


    if self.search_opt == 'grid':
      model = self.grid_opt(objective, param_grid)
      self.print_info(model)
      return model
    elif self.search_opt == 'bayesian':
      model = self.bayesian_opt(objective, param_grid)
      self.print_info(model)
      return model
    elif self.search_opt == 'randomized':
      model = self.randomized_opt(objective, param_grid)
      self.print_info(model)
      return model
    elif self.search_opt == 'none':
      scores = cross_validate(objective, self.X_train, self.y_train, cv=self.cv, return_estimator=True)
      print(scores['test_score'].max())
      return scores['estimator'][scores['test_score'].argmax()]
    else:
      raise ValueError('please provide a correct search optimization technique') 


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

X = pd.read_csv('/content/drive/Shared drives/celdum/X.csv')
y = pd.read_csv('/content/drive/Shared drives/celdum/y.csv') 
del y['Unnamed: 0']
del X['Unnamed: 0']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, stratify=y)


In [ ]:
tuner = Tuner(X_train,y_train, n_iteration=1500, search_opt='bayesian')

In [ ]:
LGBM = tuner.fit_LGBM()
rfcpred = LGBM.predict(X_test)
print(LGBM.best_params_)
accuracy = accuracy_score(y_test, rfcpred)
print(accuracy)

Optimizing LGBM algorithm...
Fitting 5 folds for each of 1500 candidates, totalling 7500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   15.7s
[Parallel(n_jobs=-1)]: Done 162 tasks      | elapsed:   42.1s
[Parallel(n_jobs=-1)]: Done 365 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 648 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 1032 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 1562 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done 2119 tasks      | elapsed:  6.6min
[Parallel(n_jobs=-1)]: Done 2854 tasks      | elapsed:  8.9min
[Parallel(n_jobs=-1)]: Done 3595 tasks      | elapsed: 11.0min
[Parallel(n_jobs=-1)]: Done 4484 tasks      | elapsed: 13.7min
[Parallel(n_jobs=-1)]: Done 5423 tasks      | elapsed: 17.0min
[Parallel(n_jobs=-1)]: Done 6454 tasks      | elapsed: 19.8min
[Parallel(n_jobs=-1)]: Done 7500 out of 7500 | elapsed: 22.9min finished


0.9028639782080891
{'subsample': 0.8, 'reg_lambda': 1.5, 'reg_alpha': 1.1, 'num_leaves': 100, 'n_estimators': 100, 'min_data_in_leaf': 10, 'min_child_weight': 0.001, 'learning_rate': 0.05, 'colsample_bytree': 1.0}
{'subsample': 0.8, 'reg_lambda': 1.5, 'reg_alpha': 1.1, 'num_leaves': 100, 'n_estimators': 100, 'min_data_in_leaf': 10, 'min_child_weight': 0.001, 'learning_rate': 0.05, 'colsample_bytree': 1.0}
0.8509346928866229


In [ ]:
XG = tuner.fit_XGB()
rfcpred = XG.predict(X_test)
print(XG.best_params_)
accuracy = accuracy_score(y_test, rfcpred)
print(accuracy)

Optimizing XGB algorithm...
Fitting 5 folds for each of 1500 candidates, totalling 7500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   36.1s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done 644 tasks      | elapsed:  8.6min
[Parallel(n_jobs=-1)]: Done 1009 tasks      | elapsed: 15.4min
[Parallel(n_jobs=-1)]: Done 1454 tasks      | elapsed: 20.8min
[Parallel(n_jobs=-1)]: Done 1991 tasks      | elapsed: 27.8min
[Parallel(n_jobs=-1)]: Done 2598 tasks      | elapsed: 36.5min
[Parallel(n_jobs=-1)]: Done 3287 tasks      | elapsed: 46.5min
[Parallel(n_jobs=-1)]: Done 4056 tasks      | elapsed: 58.3min
[Parallel(n_jobs=-1)]: Done 4939 tasks      | elapsed: 71.7min
[Parallel(n_jobs=-1)]: Done 5874 tasks      | elapsed: 85.8min
[Parallel(n_jobs=-1)]: Done 6887 tasks      | elapsed: 102.0min
[Parallel(n_jobs=-1)]: Done 7500 out of 7500 | elapsed: 109.7min finished


0.9014784153490851
{'subsample': 0.7, 'silent': False, 'reg_lambda': 1.0, 'n_estimators': 30, 'min_child_weight': 3.0, 'max_depth': 20, 'learning_rate': 0.15, 'gamma': 1.0, 'colsample_bytree': 0.9, 'colsample_bylevel': 0.8}
{'subsample': 0.7, 'silent': False, 'reg_lambda': 1.0, 'n_estimators': 30, 'min_child_weight': 3.0, 'max_depth': 20, 'learning_rate': 0.15, 'gamma': 1.0, 'colsample_bytree': 0.9, 'colsample_bylevel': 0.8}
0.8543335761107065


In [ ]:
GB = tuner.fit_GBM()
rfcpred = GB.predict(X_test)
print(GB.best_params_)
accuracy = accuracy_score(y_test, rfcpred)
print(accuracy)

Optimizing Gradient Boosting algorithm...
Fitting 5 folds for each of 200 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 21.1min
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed: 31.8min
[Parallel(n_jobs=-1)]: Done 644 tasks      | elapsed: 74.4min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed: 111.7min finished


0.899323147983656
{'subsample': 0.6, 'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 6, 'learning_rate': 0.1, 'criterion': 'mae'}
{'subsample': 0.6, 'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 6, 'learning_rate': 0.1, 'criterion': 'mae'}
0.843084891154811


In [ ]:
RF = tuner.fit_RF()
rfcpred = RF.predict(X_test)
print(RF.best_params_)
accuracy = accuracy_score(y_test, rfcpred)
print(accuracy)

Optimizing Random Forest algorithm...
Fitting 5 folds for each of 500 candidates, totalling 2500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   22.8s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 644 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done 1009 tasks      | elapsed:  8.6min
[Parallel(n_jobs=-1)]: Done 1454 tasks      | elapsed: 12.6min
[Parallel(n_jobs=-1)]: Done 1981 tasks      | elapsed: 17.2min
[Parallel(n_jobs=-1)]: Done 2500 out of 2500 | elapsed: 20.5min finished


0.8976296559483627
{'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 5, 'max_features': 'log2', 'max_depth': None, 'criterion': 'gini', 'bootstrap': False}
{'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 5, 'max_features': 'log2', 'max_depth': None, 'criterion': 'gini', 'bootstrap': False}
0.8507728413045238


In [ ]:
AdaBoost = tuner.fit_AdaBoost()
rfcpred = AdaBoost.predict(X_test)
print(AdaBoost.best_params_)
accuracy = accuracy_score(y_test, rfcpred)
print(accuracy)

Optimizing AdaBoost algorithm...
Fitting 5 folds for each of 49 candidates, totalling 245 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 49 is smaller than n_iter=1500. Running 49 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   35.1s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 245 out of 245 | elapsed:  4.0min finished


0.8865442056019421
{'n_estimators': 700, 'learning_rate': 0.3}
{'n_estimators': 700, 'learning_rate': 0.3}
0.8621024520514688


In [ ]:
ANN = tuner.fit_ANN()
rfcpred = ANN.predict(X_test)
print(ANN.best_params_)
accuracy = accuracy_score(y_test, rfcpred)
print(accuracy)

Optimizing Artificial Neural Network...
Fitting 5 folds for each of 200 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   40.5s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed:  9.1min
[Parallel(n_jobs=-1)]: Done 644 tasks      | elapsed: 14.6min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed: 25.5min finished


0.8967053946823
{'solver': 'sgd', 'learning_rate_init': 0.1, 'learning_rate': 'constant', 'hidden_layer_sizes': (200, 200), 'batch_size': 300, 'alpha': 1.5, 'activation': 'relu'}
{'solver': 'sgd', 'learning_rate_init': 0.1, 'learning_rate': 'constant', 'hidden_layer_sizes': (200, 200), 'batch_size': 300, 'alpha': 1.5, 'activation': 'relu'}
0.8339402767662054


In [ ]:
SVM = tuner.fit_SVM()
rfcpred = SVM.predict(X_test)
print(SVM.best_params_)
accuracy = accuracy_score(y_test, rfcpred)
print(accuracy)

Optimizing SVM algorithm...
Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   27.0s


In [ ]:
Bagging = tuner.fit_Bagging()
rfcpred = Bagging.predict(X_test)
print(Bagging.best_params_)
accuracy = accuracy_score(y_test, rfcpred)
print(accuracy)

Optimizing Bagging algorithm...
Fitting 5 folds for each of 35 candidates, totalling 175 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 35 is smaller than n_iter=1500. Running 35 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   22.7s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 175 out of 175 | elapsed:  2.7min finished


0.899169538698407
{'n_estimators': 250, 'max_samples': 0.2}
{'n_estimators': 250, 'max_samples': 0.2}
0.8506919155134741


In [ ]:
LOGR = tuner.fit_LogR()
rfcpred = LOGR.predict(X_test)
print(LOGR.best_params_)
accuracy = accuracy_score(y_test, rfcpred)
print(accuracy)

Optimizing Logistic Regression algorithm...
Fitting 5 folds for each of 14 candidates, totalling 70 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 14 is smaller than n_iter=1500. Running 14 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


0.8806950908983241
{'penalty': 'l2', 'C': 1}
{'penalty': 'l2', 'C': 1}
0.8483450675730355


[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:    1.1s finished


In [ ]:
DT = tuner.fit_DT()
rfcpred = DT.predict(X_test)
print(DT.best_params_)
accuracy = accuracy_score(y_test, rfcpred)
print(accuracy)

Optimizing Decision Tree algorithm...
Fitting 5 folds for each of 1500 candidates, totalling 7500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 358 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 4230 tasks      | elapsed:   14.3s


0.8837759223071002
{'min_samples_split': 3, 'min_samples_leaf': 10, 'max_features': 'auto', 'max_depth': 15, 'criterion': 'entropy'}
{'min_samples_split': 3, 'min_samples_leaf': 10, 'max_features': 'auto', 'max_depth': 15, 'criterion': 'entropy'}
0.8557902403495994


[Parallel(n_jobs=-1)]: Done 7500 out of 7500 | elapsed:   24.6s finished


In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
print(classification_report(y_test, rfcpred, digits=4))
print(roc_auc_score(y_test, rfcpred))


              precision    recall  f1-score   support

           0     0.9892    0.8429    0.9102     10965
           1     0.4283    0.9274    0.5860      1392

    accuracy                         0.8524     12357
   macro avg     0.7088    0.8852    0.7481     12357
weighted avg     0.9260    0.8524    0.8737     12357

0.8851530929131878
